
# Time Series with Prodes Dataset
[Prodes dataset](https://data.globalforestwatch.org/datasets/gfw::prodes-deforestation-in-amazonia/about)

By Stephanie Mennear

## Cleaning the data

The Prodes data set contains many data points. From observing the dataset, most of the satellite images are taken from August - September, which are the months of the year that the Amazon is not mostly covered by clouds. It is during this time, that the satellite images can be taken for evaluation.

Because the clouds obstruct the image of the satellite data, there are no consistent patterns for date ranges that the data is collected.

To clean the data, the following functions drop all columns except the columns that hold values for States, date the images were taken, and the areakm.
After looking at the satellite images, I came to the discovery that the areakm data points detailed the measurment of deforestation from the previous year. Therefore, after the data is cleaned, the cummulative sum is the value the time series forecast model will use.

In order to prepare the dataset for the time series forecast model, the data must be prepared with a datetime index, as well as the data points to fit the date time index. Because of the inconsistant data points (due to cloud cover), the redisturbution method works as follows:

1. The data points for each specific state of each specific year are redistributed to fit 365 days.
2. The left over rows that did not fit into 365 days are summed, and divided amoung the 356 rows evenly. This means- each row value is added with the divided remaining value. This was to keep the integrity of the total amount of areakm that was deforested, and to prepare the dataset for yearly forecasting.
3. Due to the method that it was distubuted, no daily, or monthly trends should be observed. Only yearly.


## Prepare environment 
## Cleaning data


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.filters.hp_filter import hpfilter
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.ar_model import AR, ARResults
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import mse, rmse, meanabs
!pip install pmdarima
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA, ARMA, ARMAResults, ARIMAResults



- sum all the km for every year. 
- First, organize by year- collect dfs of each year. 
- second, get sum for each year. 


In [ ]:
df = pd.read_csv('PRODES_Deforestation_in_Amazonia.csv')

In [ ]:
def df_clean(df_):
    df_.dropna(inplace=True)
    df_.drop(columns=['FID', 'ORIGIN_ID', 'PATH_ROW', 'DEF_CLOUD', 'SCENE_ID', 'PUBLISH_YE', 'SOURCE', 'SATELLITE', 'SENSOR', 'MAIN_CLASS', 'CLASS_NAME', 'JULIAN_DAY', 'ID'], inplace=True)    
    df_.sort_values(by='IMAGE_DATE', inplace=True)
    df_['IMAGE_DATE'] = pd.to_datetime(df_['IMAGE_DATE'], format='%Y-%m-%d', errors='coerce')
    df_.set_index(df_['IMAGE_DATE'], inplace=True)
    df_.dropna(inplace=True)
    org_yearly(df_)

In [ ]:
def org_yearly(df_):
  df_name = {}
  for year in df_.index.year.unique():
    print('DF for {}'.format(year))
    df_name[year] = df_['{}'.format(year)]
    print(df_name[year])

In [ ]:
df_clean(df)

### Preparing for time models

   - resample yearly
   - Add cumsum()

In [ ]:
df_y = df.resample('A').sum()

In [ ]:
df_y['cumsum'] = df_y.AREA_KM.cumsum()

In [ ]:
df_y['cumsum'].plot()

### Time Series

In [ ]:
df_y.index

In [ ]:
train_yearly = df_y.iloc[:9]
test_yearly = df_y.iloc[8:]

In [ ]:
fitted_model = ExponentialSmoothing(train_yearly['cumsum'], trend='mul', seasonal_periods=1).fit()
test_predictions = fitted_model.forecast(10)

In [ ]:
train_yearly['cumsum'].plot(figsize=(12,5), legend=True, label='Train')
test_yearly['cumsum'].plot(legend=True, label='Test')
test_predictions.plot(legend=True, label='Prediction')
plt.title('Predictions from 2017 - 2026 \nAreakm of total deforestation \n All 9 State of Legal Amazonia', fontsize=20)
plt.xlabel('Years', fontsize=12)
plt.ylabel('Areakm total deforestation', fontsize=12)


In [ ]:
test_predictions

In [ ]:
df_y['cumsum']

- put text preditions in a dataframe to download and send to flask server
<br>
- also add non predictions in a csv

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
from google.colab import files

In [ ]:
recorded_areakm = df_y['cumsum'].to_frame()
recorded_areakm.reset_index(level=0, inplace=True)
recorded_areakm.columns=['Recorded Years', 'Recorded Areakm Deforested']

In [ ]:
RecordedAreakm = recorded_areakm.to_csv()
with open('RecordedAreakm.csv', 'a') as f:
  f.write(RecordedAreakm)
files.download('RecordedAreakm.csv')

In [ ]:
test_predictions_df=test_predictions.to_frame()
test_predictions_df.reset_index(level=0, inplace=True)
test_predictions_df.columns=['Prediction Year', 'Predicted Areakm Deforested']

In [ ]:
Brazil_ESM = test_predictions_df.to_csv()
with open('BrazilESM.csv', 'a') as f:
  f.write(Brazil_ESM)

In [ ]:
files.download('BrazilESM.csv')


#### Failed model:

In [ ]:
result_y = seasonal_decompose(df_y['cumsum'], model='multiplicative')

In [ ]:
result_y.plot();

In [ ]:
auto_arima(df_y['cumsum']).summary() #m is for the seasonal periods
#SARIMAX(0, 1, 0)

In [ ]:
train_sy = df_y.iloc[:10]
test_sy = df_y.iloc[10:]

In [ ]:
model_sy = SARIMAX(train_sy['cumsum'], order=(0,1,0))
result_sy = model_sy.fit()
start_sy = len(train_sy)
end_sy = len(train_sy) + len(test_sy) -1

In [ ]:
prediction_sy = result_sy.predict(start_sy, end_sy, type='levels').rename('SARIMAX Yearly Predictions')


In [ ]:
test_sy['cumsum'].plot(legend= True, figsize=(12, 8))
prediction_sy.plot(legend=True)

#### Visualizations

#### Rate of deforestation over time from 2008 - 2019

In [ ]:
df.median()
df.describe()
df['STATE'].value_counts()

In [ ]:
state_group = df.groupby(['STATE'])
state_group['AREA_KM'].agg(['median', 'mean', 'sum'])

In [ ]:
state_group.get_group('AM')

In [ ]:
group_year = df.groupby(['IMAGE_DATE'])
group_year.get_group(2008)

In [ ]:
group_year['STATE'].value_counts()

In [ ]:
group_year['AREA_KM'].sum() #.plot

In [ ]:
grp = df.groupby(by=['YEAR', 'STATE'])

In [ ]:
grp['AREA_KM'].sum()

In [ ]:
grp['AREA_KM'].sum().plot.bar()


- The above visualizes the years 2008-2019. It is not clear by the marks, so for readability, each group of peaks represents a year. 
- From this visualization, 2008 is extremley high, while trends dip from 2009 - 2017.
- Discovering from research, Brazil saw a decline in deforestation rates from 2008 to around 2018.
  According to an article published November 2020 by [BBC](https://www.bbc.com/news/world-latin-america-55130304), in collaboration with Prodes data:
  > "Amazon deforestation highest since 2008"

Why? [BBC](https://www.bbc.com/news/world-latin-america-55130304) states:

> "Scientists say it has suffered losses at an accelerated rate since Jair Bolsonaro took office in January 2019.
> The Brazilian president has encouraged agriculture and mining activities in the world's largest rainforest."

#### Rate of deforestation by state in total from 2008-2019

In [ ]:
state_area = df.groupby(by='STATE')['AREA_KM'].sum().sort_values(ascending=False).reset_index()
state_area = state_area.sort_values(by='AREA_KM', ascending=True)

Next step is to figure out size of each state and find the percentage that it has been deforested, then to look again at the rates of reforestation

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot(x=state_area['STATE'], y=state_area['AREA_KM'], palette='Greens', alpha=0.85)
plt.title("Deforestation by State", fontsize = 25)
plt.xlabel("State", fontsize = 20)
plt.ylabel("Sum of km2 deforestation", fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.legend(fontsize = 15)

**A similar process can be seen under the Failed Methods section: "Visualization of total cumsum deforested per state"**



# Failed Methods

## Ceaning the data
**A way of cleaning data, before fully understanding the capabilites of pandas dataframe methods offered**... 

In [ ]:
df = pd.read_csv('PRODES_Deforestation_in_Amazonia.csv')
def df_clean(df_):
    df_.dropna(inplace=True)
    df_.drop(columns=['FID', 'ORIGIN_ID', 'PATH_ROW', 'DEF_CLOUD', 'SCENE_ID', 'PUBLISH_YE', 'SOURCE', 'SATELLITE', 'SENSOR', 'MAIN_CLASS', 'CLASS_NAME', 'JULIAN_DAY', 'ID'], inplace=True)    
    df_.sort_values(by='IMAGE_DATE', inplace=True)
    org_state(df_) #next function
#organize the df to clean for processing.


In [ ]:
#organize the df into states
#the individual df of indv. states will pass through to the next function, bringing the state name with it
def org_state(df_):
    df_.dropna(inplace=True) #put it here for further caution
    df_name = {}
    for state in df_.STATE.unique():       
        df_name[state] = df_.loc[df_['STATE'] == state].copy()
        print('********************', '\n', 'Information for state of {} :'.format(state), '\n', '********************', '\n')
        clean_date(df_name[state], state) #next function
        

In [ ]:
#change the Image_date to datetime
#individual df of the individual year will go through to the next function
#here a date range is created as well, this is carried through to the next function
def clean_date(df_, state):
    df_.dropna(inplace=True)
    df_year = {}
    state = state
    #turn image_date to a time series to get information about it and seperate it
    df_['IMAGE_DATE'] = pd.to_datetime(df_['IMAGE_DATE'], format='%Y-%m-%d', errors='coerce').copy()
    #time to organize the datetimes into seperate dfs. for each year
    df_.set_index(df_['IMAGE_DATE'], inplace=True)
    df_.dropna(inplace=True)   
    for year in df_.index.year.unique():
      print('*************** CLEANING FOR {}'.format(year), '\n')
      year = int(year)
      df_year[year] = df_.loc[df_.YEAR == year].copy()
      global date_rng
      date_rng = pd.date_range(start='{}-01-01'.format(year), end='{}-12-31'.format(year), freq='D') #int year to sove the float problem
      #print('date range length: ', len(date_rng))
      apply_new_dates(df_year[year], date_rng, year, state)
      



In [ ]:
#the individual state/year df will be processed to disperse across the date_range.
def apply_new_dates(df_, date_ranges, year, state):
  df_.dropna(inplace=True)
  unique_df_id = '{}'.format(state) + '{}'.format(year)
  global cleaned_date_list 
  cleaned_date_list = []
  global popped_numbers 
  popped_numbers = []
  df_Htime = {}
  subtract = {}
  Y = {}
  series_area = df_['AREA_KM']
  if(len(series_area) > len(date_ranges)):
    subtract_by = len(series_area) - len(date_ranges)
    #print(subtract_by)
    poped_rows = series_area[len(date_ranges):] #all the rows that cant fit to date range
    rows_hourly = series_area[:(len(date_ranges))] #all the rows that fit in date range
    #average = sum(poped_rows)/len(poped_rows)
    average = sum(poped_rows) #add all the values together
    dispurse_number = average / len(rows_hourly) #divide by rows in the timerange #len(poped_rows)
    lst_date_rng = date_rng.tolist()
    hourly_list = rows_hourly.tolist()
    df_Htime[unique_df_id] = pd.DataFrame({'date':lst_date_rng, 'areakm':hourly_list, 'state':state})
    disperse_areakms(df_Htime[unique_df_id], dispurse_number) #next function to disperse the left over numbers
    create_big_csv(df_Htime[unique_df_id]) #next function to add it to a file
  else:
    area_sum = df_['AREA_KM'].sum()
    dispurse_number = area_sum / len(date_ranges)
    #print('Date range length: ', len(date_ranges))
    #print('DF length: ', len(series_area))
    #series_dates = date_ranges.to_series() #in order to merge into a df
    lst_date_rng = date_ranges.to_list() #change it to list instead so index does not come with it
    df_Htime[unique_df_id] = pd.DataFrame({'date':lst_date_rng, 'areakm':0, 'state':state})
    df_Htime[unique_df_id].fillna(0, inplace=True)
    #df_Htime[unique_df_id]['areakm'] = df_Htime[unique_df_id]['areakm'] + disperser
    #send it to the function call instead
    disperse_areakms(df_Htime[unique_df_id], dispurse_number) #next function to disperse the left over numbers 
    create_big_csv(df_Htime[unique_df_id]) #next function to add it to a file
 




In [ ]:
def disperse_areakms(df_, disperser):
  df_['areakm'] = df_['areakm'] + disperser
  print(df_.head())
  return(df_)


In [ ]:
df_clean(df)

In [ ]:
#Load the csv into a dataframe
#Take headers out
#shift first column out of header column
#rename the headers
#set index
df2 = pd.read_csv('df_dattime_byStateYears1.csv', header=None)
df2.rename(columns={0:'date', 1:'areakm', 2:'state'}, inplace=True)
df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d', errors='coerce')
df2.set_index(df2['date'], inplace=True)

In [ ]:
df2.isnull().values.sum()
df2.isna().values.sum()
df2.state.unique()

In [ ]:
#all states are complete and in order of time. 
df_pa = df2.loc[df2['state'] == 'PA'] 
df_am = df2.loc[df2['state'] == 'AM'] 
df_rr = df2.loc[df2['state'] == 'RR'] 
df_ro = df2.loc[df2['state'] == 'RO'] 
df_mt = df2.loc[df2['state'] == 'MT'] 
df_ac = df2.loc[df2['state'] == 'AC'] 
df_ma = df2.loc[df2['state'] == 'MA'] 
df_to = df2.loc[df2['state'] == 'TO'] 
df_ap = df2.loc[df2['state'] == 'AP'] 

## Visualization of total cumsum deforested per state

In [ ]:
df_pa['areasum'] = df_pa.areakm.cumsum()
df_am['areasum'] = df_am.areakm.cumsum()
df_rr['areasum'] = df_rr.areakm.cumsum() 
df_ro['areasum'] = df_ro.areakm.cumsum()
df_mt['areasum'] = df_mt.areakm.cumsum()
df_ac['areasum'] = df_ac.areakm.cumsum()
df_ma['areasum'] = df_ma.areakm.cumsum() 
df_to['areasum'] = df_to.areakm.cumsum() 
df_ap['areasum'] = df_ap.areakm.cumsum() 

In [ ]:
#make a bif df with all states once again
frames = [df_pa, df_am, df_rr, df_ro, df_mt, df_ac, df_ma, df_to, df_ap]
df_cs = pd.concat(frames)

In [ ]:
areasum_states = df_cs.groupby(by='state')['areasum'].sum().sort_values(ascending=False).reset_index()
areasum_states = areasum_states.sort_values(by='areasum', ascending=True)


In [ ]:
plt.figure(figsize = (16, 9))

# plot
ax = sns.barplot(x = areasum_states['state'], y = areasum_states['areasum'], palette = "Reds", alpha = 0.85)


plt.title("States", fontsize = 25)
plt.xlabel("State", fontsize = 20)
plt.ylabel("areasum", fontsize = 20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.legend(fontsize = 15)

In [ ]:
sns.lineplot(data=df_cs, x='date', y='areasum', hue='state')

In [ ]:
fig, axs = plt.subplots(3, 3, sharex='col', sharey='row')
axs[0, 0].plot(df_pa['date'], df_pa['areasum'])
axs[0, 0].set_title('PA')
axs[0, 1].plot(df_am['date'], df_am['areasum'])
axs[0,1].set_title('AM')
axs[0, 2].plot(df_rr['date'], df_rr['areasum'])
axs[0,2].set_title('RR')
axs[1, 0].plot(df_ro['date'], df_ro['areasum'])
axs[1,0].set_title('RO')
axs[1, 1].plot(df_mt['date'], df_mt['areasum'])
axs[1, 1].set_title('MT')
axs[1, 2].plot(df_ac['date'], df_ac['areasum'])
axs[1, 2].set_title('AC')
axs[2, 0].plot(df_ma['date'], df_ma['areasum'])
axs[2,0].set_title('MA')
axs[2, 1].plot(df_to['date'], df_to['areasum'])
axs[2,1].set_title('TO')
axs[2, 2].plot(df_ap['date'], df_ap['areasum'])
axs[2,2].set_title('AP')


## Failed Models

Testing models on the state of Amazonia over the years of 2008-2019. df_am


### Resampling, monthly

In [ ]:
df_am_month = df_am.resample(rule='MS').mean()

In [ ]:
train_monthly = df_am_month.iloc[:121]
test_monthly = df_am_month.iloc[120:]

In [ ]:
fitted_model = ExponentialSmoothing(train_monthly['areasum'], trend='add', seasonal_periods=12).fit()
test_predictions = fitted_model.forecast(24) #2 years into future

In [ ]:
#plot them all
train_monthly['areasum'].plot(figsize=(12,5), legend=True, label='Train')
test_monthly['areasum'].plot(legend=True, label='Test')
test_predictions.plot(legend=True, label='Prediction')


### Yearly

In [ ]:
df_am_year = df_am.resample(rule='A').mean()
df_am_year.plot()
#areakm is white noise

In [ ]:
train_yearly = df_am_year.iloc[:9]
test_yearly = df_am_year.iloc[8:]

In [ ]:
#fitted_model = ExponentialSmoothing(train_monthly['areasum'], trend='add', seasonal_periods=12).fit()
fitted_model = ExponentialSmoothing(train_yearly['areasum'], trend='add', seasonal_periods=1).fit()
test_predictions = fitted_model.forecast(10) #2 years into future

In [ ]:
train_yearly['areasum'].plot(figsize=(12,5), legend=True, label='Train')
test_yearly['areasum'].plot(legend=True, label='Test')
test_predictions.plot(legend=True, label='Prediction')



### Sarimax with Monthly data

In [ ]:
#look at the seasonal decompose:
result_m = seasonal_decompose(df_am_month['areasum'], model='add')
result_m.plot();

In [ ]:
auto_arima(df_am_month['areasum'], seasonal=True, m=12).summary() #monthly data, 12 rows per year

In [ ]:
#forecast into future- length of dataset is 144 rows
#train_sm = train Sarimax Monthly
train_sm = df_am_month.iloc[:132] #minus the last year
test_sm = df_am_month.iloc[132:]

In [ ]:
model = SARIMAX(train_sm['areasum'], order=(0,2,0))

In [ ]:
result_sm = model.fit()
result_sm.summary()

In [ ]:
start_sm = len(train_sm)
end_sm = len(train_sm) + len(test_sm) - 1

In [ ]:
#now create the predictions
prediction_sm = result_sm.predict(start_sm, end_sm, type='levels').rename('Sarima Monthly predications')

In [ ]:
test_sm['areasum'].plot(legend=True, figsize=(12, 8))
prediction_sm.plot(legend=True)

In [ ]:
fcast = result_sm.predict(len(df_am_month), len(df_am_month) + 11, type='levels').rename('Sarima Forecast')
test_sm['areasum'].plot(legend=True, figsize=(12, 8))
prediction_sm.plot(legend=True)
fcast.plot(legend=True)

### Sarimax with Yearly data

In [ ]:
result_y = seasonal_decompose(df_am_year['areasum'], model='add')
result_y.plot();

In [ ]:
auto_arima(df_am_year['areasum'], m=1).summary()
#SARIMAX(2, 1, 0)

In [ ]:
train_sy = df_am_year.iloc[:10] #minus the last year
test_sy = df_am_year.iloc[10:]

In [ ]:
model_y = SARIMAX(train_sy['areasum'], order=(2,1,0))
resuly_y = model_y.fit()
start_sy = len(train_sy)
end_sy = len(train_sy) + len(test_sy) - 1

In [ ]:
#now create the predictions
prediction_sy = resuly_y.predict(start_sy, end_sy, type='levels').rename('Sarima Yearly predications')

In [ ]:
test_sy['areasum'].plot(legend=True, figsize=(12, 8))
prediction_sy.plot(legend=True)

In [ ]:
#future
fcast_y = resuly_y.predict(len(df_am_year), len(df_am_year) + 3, type='levels').rename('Sarima Forecast')

In [ ]:
test_sy['areasum'].plot(legend=True, figsize=(12, 8))
prediction_sy.plot(legend=True)
fcast_y.plot(legend=True)

### More failed models

Notes: 
- choosing arma/arima orders. chooseing the best p,q,a
- finding out the orders of the ar and ma components.
- finding out if the i component is needed
- if the aurocorrelation plot shows positive autocorrelation at first lag (lag-1), suggested to use AR terms in relation to lag
- if the autocorrelation plot shows negative autocorrelation, suggest using MA terms
- p: number of lag observations included in ar component of the model
- d: number of times the raw observations are differenced
- q: size of moving average window.

##Arima
non seasonal arima (p, d, q) </br>
p = corresponds to the AR poriont of model</br>
d = Indegrated componend. Differencing- diff of observations. In order to make the time series stationary (statsmodels as the diff function) </br>
q =  corresponds to MA component. Plotting out moving average, and using the residual error </br>


In [ ]:
def adf_test(series, title=''):
  print(f'Augmented Dickey-Fuller Test: {title}')
  result=adfuller(series.dropna(), autolag='AIC')
  labels=['ADF test stats', 'p-value', 'number of lags used', 'number of observations']
  out = pd.Series(result[0:4], index=labels)
  for key, val in result[4].items():
    out[f'critical value ({key})'] = val
  
  print(out.to_string())

  if result[1] <= 0.05:
    print('Strong evidence against null hypthesis.', '\n', 'Reject null hypothesis.', '\n', 'Data has no unit root and is stationary')
  else:
    print('Weake evidence against null hypthesis.', '\n', 'Fail to reject null hypothesis.', '\n', 'Data has a unit root and is non-stationary')

model = AR(train_data['areasum'])

In [ ]:
model = AR(train_data['areasum'])
AR1fit.params

In [ ]:
start = len(train_data)
end = len(train_data) + len(test_data) -1

In [ ]:
AR1fit.predict(start=start, end=end)

In [ ]:
#compare predicted values to real known test values
prediction1 = AR1fit.predict(start=start, end=end)
#name it to keep track
prediction1 = prediction1.rename('AR(1) Predictions')

In [ ]:
test_data['areasum'].plot(figsize=(12,8), legend=True, label='Origional')
prediction1.plot(legend=True)
#under predicting

In [ ]:
#try to improve this by expanding order
#name it to keep track
AR2fit = model.fit(maxlag=2)
AR2fit.params

In [ ]:
prediction2 = AR2fit.predict(start=start, end=end)
prediction2 = prediction2.rename('AR2 predictions')

In [ ]:
#now plot the three
test_data['areasum'].plot(figsize=(12,8), legend=True, label='Origional')
prediction1.plot(legend=True)
prediction2.plot(legend=True)

In [ ]:
#finidng the correct order value, let statsmodels decide, do not specify maxlag
#look into the ic parameter
ARfit = model.fit(ic='t-stat')
ARfit.params

In [ ]:
prediction28 = ARfit.predict(start=start, end=end)
prediction28 = prediction28.rename('AR28 Predictions')

In [ ]:
#evaluate it
labels = ['AR1', 'AR2', 'AR28']
preds = [prediction1, prediction2, prediction28]

In [ ]:
for i in range(3):
  #np.sqrt() #use if you want
  error = mean_squared_error(test_data['areasum'], preds[i])
  print(f'{labels[i]} error: {error}')

In [ ]:
#plot all 4
test_data['areasum'].plot(figsize=(12,8), legend=True, label='Origional')
prediction1.plot(legend=True)
prediction2.plot(legend=True)
prediction28.plot(legend=True)

#### Arma with df_am_year1

In [ ]:
#we know seasonal is false
#trace will show you the first few arima models that it is trying to fix
stepwise_fit = auto_arima(df_am['areasum'], start_P=0, start_q=0, max_p=6, max_q=3, seasonal=False, trace=True)

In [ ]:
stepwise_fit.summary()
#why does it suggest a SARIMA model? 

In [ ]:
#look at first year
df_am_year1 = df_am[:366]

In [ ]:
df_am_year1.index

In [ ]:
df_am_year1['areakm'].plot(figsize=(12,8))

In [ ]:
adf_test(df_am_year1['areakm'])

In [ ]:
auto_arima(df_am_year1['areakm'], seasonal=False).summary()

In [ ]:
auto_arima(df_am['areakm'], seasonal=False).summary()

 Failed, moving on to **ARIMA**

#### ARIMA with df_am

using df_am

In [ ]:
test_am = df_am[:3650]
train_am = df_am[3650:]

In [ ]:
model = ARIMA(train_am['areakm'], order=(2,1,3))
results = model.fit()
results = model.fit()

In [ ]:
#set start and end location (dates)
start_am = len(train_am)
end_am = len(train_am) + len(test_am) -1

In [ ]:
predictions = results.predict(start_am,end_am).rename('ARIMA(2,1,3)')

In [ ]:
test_am['areakm']

In [ ]:
predictions

In [ ]:
test_am['areakm'].plot(figsize=(12,8), legend=True)
predictions.plot(legend=True)

### Resample yearly, SARIMAX

In [ ]:
df_am_yearly = df_am.resample(rule='A').sum()

In [ ]:
df_am_yearly.plot()

In [ ]:
results3 = auto_arima(df_am_yearly, start_P=0, start_q=0, max_p=6, max_q=3, seasonal=False, trace=True)

In [ ]:
results3.summary()

- the time series is white noise?
- mean = 0
- std is contant with time
- correlatoin lag between lags is 0
- is it stationary?
- mean is constant
- std is constant
- no seasonality


In [ ]:
smodel = SARIMAX(train_am['areakm'], order=(0,1,3), enforce_invertibility=False)

In [ ]:
results = smodel.fit()
results.summary()

In [ ]:
start = len(train_am)
end = len(train_am) +len(test_am) -1


In [ ]:
predictions = results.predict(start,end).rename('Sarima')
predictions = results.predict(start,end).rename('Sarima')
test_am['areakm'].plot(figsize=(15,8))
predictions.plot()

# Evaluation Methods

###ACF and PACF
-autocorrelation and partial autocorrelation tests </br>
-correlation: closer to 0, weak relationship. +1 / -1 negativly or positively correlated

- autocorrelation, compares a series to itself, lagged

In [ ]:
#check if frequency is set
#if none, then set it
df_am.index.freq= 'D'
print(df_am.index)

In [ ]:
#auto corelation
acf(df_am['areasum'])
#positivly correlated

In [ ]:
lag_plot(df_am['areasum'])
#is this correct? this is purely strong autocorrelation

In [ ]:
plot_acf(df_am['areasum'], lags=40);
#confidence interval - correlation values outside of it, highly likely to be a correlation. 

In [ ]:
#look at the same plot on the areakm column
plot_acf(df_am['areakm'], lags=40);

In [ ]:
#what is this? 
df_am['areakm'].plot()

In [ ]:
#PACF
#stationary
#if a sharp cutoff, indicator to add pr terms
plot_pacf(df_am['areakm'], lags=40);

In [ ]:
#non stationary
plot_pacf(df_am['areasum'], lags=40);